In [1]:
import os
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from PIL import Image

# Directorio donde están las imágenes organizadas por carpetas (una por cada set)
data_dir = "../TRABAJO/FOTOS"

# Transformaciones para las imágenes
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Redimensionar todas las imágenes
    transforms.RandomHorizontalFlip(),  # Aumento de datos: reflejo horizontal
    transforms.RandomRotation(15),  # Aumento de datos: rotación aleatoria
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Ajuste de color
    transforms.ToTensor(),  # Convertir a tensor
    transforms.Normalize([0.5], [0.5])  # Normalizar valores de píxeles
])

# Cargar dataset con ImageFolder (cada carpeta es una clase)
dataset = ImageFolder(root=data_dir, transform=transform)

# Dividir en entrenamiento (80%), validación (10%) y prueba (10%)
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

# Crear DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print("Dataset preprocesado y dividido:")
print(f"- Entrenamiento: {len(train_dataset)} imágenes")
print(f"- Validación: {len(val_dataset)} imágenes")
print(f"- Prueba: {len(test_dataset)} imágenes")


Dataset preprocesado y dividido:
- Entrenamiento: 1062 imágenes
- Validación: 132 imágenes
- Prueba: 134 imágenes


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Comprobar si hay GPU disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Cargar el modelo preentrenado EfficientNetB0
model = models.efficientnet_b0(pretrained=True)

# Modificar la última capa para clasificar nuestros sets de LEGO
num_features = model.classifier[1].in_features
num_classes = len(dataset.classes)  # Número de sets de LEGO
model.classifier[1] = nn.Linear(num_features, num_classes)

# Enviar el modelo a GPU si está disponible
model = model.to(device)

# Definir la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entrenamiento del modelo
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    train_acc = 100 * correct / total
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    # Evaluación en conjunto de validación
    model.eval()
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
    
    val_acc = 100 * correct_val / total_val

    print(f"Época {epoch+1}/{num_epochs} - Pérdida: {running_loss/len(train_loader):.4f} - "
          f"Acc. Entrenamiento: {train_acc:.2f}% - Acc. Validación: {val_acc:.2f}%")

# Guardar el modelo entrenado
torch.save(model.state_dict(), "../04_Extra/ID/modelo_lego.pth")
print("Entrenamiento finalizado. Modelo guardado.")


/Users/luismgl/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/luismgl/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /Users/luismgl/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:32<00:00, 669kB/s] 


Época 1/20 - Pérdida: 3.0010 - Acc. Entrenamiento: 29.94% - Acc. Validación: 61.36%
Época 2/20 - Pérdida: 1.1246 - Acc. Entrenamiento: 73.35% - Acc. Validación: 69.70%
Época 3/20 - Pérdida: 0.5109 - Acc. Entrenamiento: 87.76% - Acc. Validación: 80.30%
Época 4/20 - Pérdida: 0.3732 - Acc. Entrenamiento: 90.77% - Acc. Validación: 76.52%
Época 5/20 - Pérdida: 0.2413 - Acc. Entrenamiento: 94.54% - Acc. Validación: 81.06%
Época 6/20 - Pérdida: 0.2301 - Acc. Entrenamiento: 94.16% - Acc. Validación: 81.06%
Época 7/20 - Pérdida: 0.1565 - Acc. Entrenamiento: 96.05% - Acc. Validación: 81.06%
Época 8/20 - Pérdida: 0.0919 - Acc. Entrenamiento: 98.31% - Acc. Validación: 81.06%
Época 9/20 - Pérdida: 0.0751 - Acc. Entrenamiento: 98.40% - Acc. Validación: 84.85%
Época 10/20 - Pérdida: 0.0764 - Acc. Entrenamiento: 98.21% - Acc. Validación: 82.58%
Época 11/20 - Pérdida: 0.1471 - Acc. Entrenamiento: 96.42% - Acc. Validación: 78.79%
Época 12/20 - Pérdida: 0.1310 - Acc. Entrenamiento: 97.27% - Acc. Validaci

In [3]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np

# Comprobar si hay GPU disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# Directorio del dataset
data_dir = "../TRABAJO/FOTOS"

# Aumento de datos y preprocesamiento
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Redimensionar imágenes
    transforms.RandomHorizontalFlip(p=0.5),  # Reflejo horizontal aleatorio
    transforms.RandomRotation(15),  # Rotación aleatoria
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Variación de color
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1)),  # Pequeños desplazamientos
    transforms.RandomPerspective(distortion_scale=0.2, p=0.5),  # Perspectiva aleatoria
    transforms.ToTensor(),  # Convertir a tensor
    transforms.Normalize([0.5], [0.5])  # Normalización
])

# Cargar dataset
dataset = ImageFolder(root=data_dir, transform=transform)
num_classes = len(dataset.classes)  # Número de sets de LEGO

# Dividir en entrenamiento (80%), validación (10%) y prueba (10%)
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Crear DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"- Imágenes de entrenamiento: {len(train_dataset)}")
print(f"- Imágenes de validación: {len(val_dataset)}")
print(f"- Imágenes de prueba: {len(test_dataset)}")

# Cargar modelo preentrenado EfficientNetB0
model = models.efficientnet_b0(pretrained=True)

# Modificar la última capa para clasificar nuestros sets de LEGO
num_features = model.classifier[1].in_features
model.classifier = nn.Sequential(
    nn.Dropout(0.5),  # Dropout para evitar sobreajuste
    nn.Linear(num_features, num_classes)
)

# Enviar modelo a GPU si está disponible
model = model.to(device)

# Definir función de pérdida y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Scheduler para reducir el LR si la validación no mejora
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

# Early Stopping manual
early_stopping_patience = 5
best_val_loss = np.inf
epochs_no_improve = 0

# Entrenamiento del modelo
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()
    
    train_acc = 100 * correct_train / total_train
    avg_train_loss = running_loss / len(train_loader)

    # Evaluación en validación
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
    
    val_acc = 100 * correct_val / total_val
    avg_val_loss = val_loss / len(val_loader)

    # Reducir el LR si la validación no mejora
    scheduler.step(avg_val_loss)

    # Early stopping
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), "../04_Extra/ID/modelo_lego_final.pth")  # Guardar el mejor modelo
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= early_stopping_patience:
            print(f"Early stopping activado en la época {epoch+1}")
            break

    print(f"Época {epoch+1}/{num_epochs} - "
          f"Pérdida: {avg_train_loss:.4f} - Acc. Entrenamiento: {train_acc:.2f}% - "
          f"Acc. Validación: {val_acc:.2f}% - Pérdida Validación: {avg_val_loss:.4f}")

print("Entrenamiento finalizado. Modelo guardado como 'modelo_lego_final.pth'.")


Usando dispositivo: cpu
- Imágenes de entrenamiento: 1062
- Imágenes de validación: 132
- Imágenes de prueba: 134


/Users/luismgl/Library/Python/3.9/lib/python/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Época 1/30 - Pérdida: 3.3045 - Acc. Entrenamiento: 19.96% - Acc. Validación: 44.70% - Pérdida Validación: 2.4201
Época 2/30 - Pérdida: 1.6719 - Acc. Entrenamiento: 59.51% - Acc. Validación: 70.45% - Pérdida Validación: 1.3222
Época 3/30 - Pérdida: 0.8783 - Acc. Entrenamiento: 78.34% - Acc. Validación: 74.24% - Pérdida Validación: 1.0850
Época 4/30 - Pérdida: 0.5218 - Acc. Entrenamiento: 86.06% - Acc. Validación: 78.03% - Pérdida Validación: 1.0551
Época 5/30 - Pérdida: 0.3416 - Acc. Entrenamiento: 91.71% - Acc. Validación: 80.30% - Pérdida Validación: 0.9184
Época 6/30 - Pérdida: 0.3638 - Acc. Entrenamiento: 90.30% - Acc. Validación: 72.73% - Pérdida Validación: 1.1852
Época 7/30 - Pérdida: 0.3345 - Acc. Entrenamiento: 91.34% - Acc. Validación: 75.76% - Pérdida Validación: 1.1977
Época 8/30 - Pérdida: 0.2484 - Acc. Entrenamiento: 93.79% - Acc. Validación: 81.82% - Pérdida Validación: 0.8905
Época 9/30 - Pérdida: 0.2322 - Acc. Entrenamiento: 93.50% - Acc. Validación: 78.03% - Pérdida Va

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np

# 🚀 Elegir modelo (EfficientNetB0 o ResNet50)
MODEL_NAME = "efficientnet"

# 🔥 Configuración del dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# 🔥 Directorio del dataset
data_dir = "../TRABAJO/FOTOS"

# 🔥 Data Augmentation Mejorado
transform = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.2)),  # Variabilidad de tamaño
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(30),
    transforms.ColorJitter(brightness=0.6, contrast=0.6, saturation=0.6, hue=0.4),
    transforms.RandomAffine(degrees=30, translate=(0.2, 0.2)),
    transforms.RandomPerspective(distortion_scale=0.5, p=0.5),
    transforms.RandomGrayscale(p=0.4),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),  # Desenfoque aleatorio
    transforms.ToTensor(),
    transforms.Normalize([0.485], [0.456], [0.406])
])

# 🔥 Cargar dataset
dataset = ImageFolder(root=data_dir, transform=transform)
num_classes = len(dataset.classes)

# 🔥 División en entrenamiento (80%), validación (10%) y prueba (10%)
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# 🔥 Crear DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"- Imágenes de entrenamiento: {len(train_dataset)}")
print(f"- Imágenes de validación: {len(val_dataset)}")
print(f"- Imágenes de prueba: {len(test_dataset)}")

# 🔥 Cargar modelo seleccionado
if MODEL_NAME == "efficientnet":
    model = models.efficientnet_b0(pretrained=True)
    num_features = model.classifier[1].in_features

    # 🔥 Transfer Learning: Congelar capas base las primeras 5 épocas
    for param in model.features.parameters():
        param.requires_grad = False

    model.classifier = nn.Sequential(
        nn.Dropout(0.6),
        nn.Linear(num_features, num_classes)
    )

model = model.to(device)

# 🔥 Optimizer y Learning Rate
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

# 🔥 Función de pérdida y reducción del LR
criterion = nn.CrossEntropyLoss()
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=2, verbose=True)

early_stopping_patience = 4
best_val_loss = np.inf
epochs_no_improve = 0
unfreeze_epoch = 5  # Descongelar capas base después de 5 épocas

# 🚀 Entrenamiento del modelo
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    
    # 🔥 Descongelar capas base después de 5 épocas
    if epoch == unfreeze_epoch:
        print("🔓 Descongelando capas base...")
        for param in model.features.parameters():
            param.requires_grad = True
        optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)  # Reducir LR

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()
    
    train_acc = 100 * correct_train / total_train
    avg_train_loss = running_loss / len(train_loader)

    # 🔥 Evaluación en validación
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
    
    val_acc = 100 * correct_val / total_val
    avg_val_loss = val_loss / len(val_loader)

    scheduler.step(avg_val_loss)

    # 🔥 Early Stopping
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), "../04_Extra/ID/modelo_lego_final.pth")
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= early_stopping_patience:
            print(f"🚨 Early stopping activado en la época {epoch+1}")
            break

    print(f"Época {epoch+1}/{num_epochs} - "
          f"Pérdida Entrenamiento: {avg_train_loss:.4f} - Acc. Entrenamiento: {train_acc:.2f}% - "
          f"Pérdida Validación: {avg_val_loss:.4f} - Acc. Validación: {val_acc:.2f}%")

print("🎉 Entrenamiento finalizado. Modelo guardado como 'modelo_lego_final.pth'.")


Usando dispositivo: cpu
- Imágenes de entrenamiento: 1062
- Imágenes de validación: 132
- Imágenes de prueba: 134


/Users/luismgl/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/luismgl/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/Users/luismgl/Library/Python/3.9/lib/python/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Época 1/30 - Pérdida Entrenamiento: 3.9686 - Acc. Entrenamiento: 4.14% - Pérdida Validación: 3.8552 - Acc. Validación: 5.30%
Época 2/30 - Pérdida Entrenamiento: 3.6987 - Acc. Entrenamiento: 6.87% - Pérdida Validación: 3.6807 - Acc. Validación: 12.12%
Época 3/30 - Pérdida Entrenamiento: 3.5373 - Acc. Entrenamiento: 12.99% - Pérdida Validación: 3.4246 - Acc. Validación: 14.39%
Época 4/30 - Pérdida Entrenamiento: 3.3987 - Acc. Entrenamiento: 16.38% - Pérdida Validación: 3.3707 - Acc. Validación: 17.42%
Época 5/30 - Pérdida Entrenamiento: 3.2620 - Acc. Entrenamiento: 16.67% - Pérdida Validación: 3.3021 - Acc. Validación: 19.70%
🔓 Descongelando capas base...
Época 6/30 - Pérdida Entrenamiento: 3.1297 - Acc. Entrenamiento: 20.34% - Pérdida Validación: 3.2396 - Acc. Validación: 23.48%
Época 7/30 - Pérdida Entrenamiento: 3.0409 - Acc. Entrenamiento: 25.05% - Pérdida Validación: 3.0874 - Acc. Validación: 25.00%
Época 8/30 - Pérdida Entrenamiento: 2.8796 - Acc. Entrenamiento: 26.46% - Pérdida Va